In [19]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pandas import DataFrame
from subprocess import call
from six.moves import urllib
import os.path
import re
import sys
import tarfile
from scipy.stats.stats import pearsonr
import operator

import matplotlib

### constants

In [20]:
folders = ['/home/payam/dataset/stanford_mobile_images/cd_covers/']
data_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
model_directory = '/home/payam/workspace/models/'

### Get Model

In [21]:
file_name = data_url.split('/')[-1]
file_name = model_directory + file_name
if not os.path.exists(file_name):
    file_name, _ = urllib.request.urlretrieve(data_url, file_name)

In [22]:
tarfile.open(file_name, 'r:gz').extractall(model_directory)

### create graph

In [23]:
with tf.gfile.FastGFile(os.path.join(model_directory, 'classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
_ = tf.import_graph_def(graph_def, name='')

### extract function

In [30]:
def extract(image):
    if not tf.gfile.Exists(image):
        tf.logging.fatal('File does not exist %s', image)
    image_data = tf.gfile.FastGFile(image, 'rb').read()
    
    with tf.Session() as sess:
        # Some useful tensors:
        # 'softmax:0': A tensor containing the normalized prediction across
        #   1000 labels.
        # 'pool_3:0': A tensor containing the next-to-last layer containing 2048
        #   float description of the image.
        # 'DecodeJpeg/contents:0': A tensor containing a string providing JPEG
        #   encoding of the image.
        # Runs the softmax tensor by feeding the image_data as input to the graph.
        tensor = sess.graph.get_tensor_by_name('mixed_9/conv:0')
        features = sess.run(tensor,{'DecodeJpeg/contents:0': image_data})
    return features

In [31]:
folder_name = folders[0] + 'Reference/'
output_file = "{0}features{1}.txt".format(folder_name, '_test1')
with open(output_file, 'w') as writer:
    for i in range(1,101):
        image = folder_name + str(i).zfill(3) + '.jpg'
        #print image
        features = extract(image)
        np.savetxt(writer, features.reshape(1,-1), fmt='%.8g')
        break;
train_file = output_file

folder_name = folders[0] + 'Canon/'
output_file = "{0}features{1}.txt".format(folder_name, '_test1')
with open(output_file, 'w') as writer:
    for i in range(1,101):
        image = folder_name + str(i).zfill(3) + '.jpg'
        #print image
        features = extract(image)
        np.savetxt(writer, features.reshape(1,-1), fmt='%.8g')
        break;
test_file = output_file

### testing results

In [32]:
train = np.loadtxt(train_file)
test = np.loadtxt(test_file)

In [33]:
def calculate_accuracy( ref, test):
    pred = prediction(ref, test)
    hit_count = 0.0
    for i in range(len(test)):
        if i == pred[i]:
            hit_count += 1
    return hit_count / len(test) * 100

def prediction_window( ref, test):
    pear = {}
    corr = {}
    dist = {}
    for i in range(len(test)):
        p = {}
        c = {}
        d = {}
        for j in range(0,len(ref)):
            p[j] = abs(pearsonr(ref[j], test[i])[0])
            c[j] = np.correlate(ref[i], test[j])
            d[j] = np.linalg.norm(ref[i] - test[j])
        pear[i] = sorted(p.items(), key=operator.itemgetter(1), reverse = True)
        corr[i] = sorted(c.items(), key=operator.itemgetter(1), reverse = True)
        dist[i] = sorted(d.items(), key=operator.itemgetter(1))
    return [pear, corr, dist]

def prediction_using_combined_distance_window(pear, corr, dist):
    pred = []
    for i in range(len(pear)):
        res = np.zeros(len(pear))
        p = pear[i]
        c = corr[i]
        d = dist[i]
        weight = 10
        for j in range(0,10):
            (pi, v) = p[j]
            (ci, v) = c[j]
            (di, v) = d[j]
            res[pi] += weight * 46
            res[ci] += weight * 25
            res[di] += weight * 30
            weight -= 1
        pred.append(np.argmax(res))
    return pred

def prediction( ref, test):
    [pear, corr, dist] = prediction_window(ref, test)
    return prediction_using_combined_distance_window(pear, corr, dist)

In [34]:
res = calculate_accuracy(train, test)

In [35]:
res

22.0

In [13]:
sess = tf.Session()

In [37]:
image_data = tf.gfile.FastGFile(image, 'rb').read()
tensor = sess.graph.get_tensor_by_name('mixed_9/conv:0')
features = sess.run(tensor,{'DecodeJpeg/contents:0': image_data})

In [41]:
import matplotlib.pyplot as plt
%matplotlib inline

len(image_data)

2151394

In [18]:
for op in sess.graph.get_operations():
    print op.values()

(<tf.Tensor 'DecodeJpeg/contents:0' shape=() dtype=string>,)
(<tf.Tensor 'DecodeJpeg:0' shape=(?, ?, 3) dtype=uint8>,)
(<tf.Tensor 'Cast:0' shape=(?, ?, 3) dtype=float32>,)
(<tf.Tensor 'ExpandDims/dim:0' shape=(1,) dtype=int32>,)
(<tf.Tensor 'ExpandDims:0' shape=(1, ?, ?, 3) dtype=float32>,)
(<tf.Tensor 'ResizeBilinear/size:0' shape=(2,) dtype=int32>,)
(<tf.Tensor 'ResizeBilinear:0' shape=(1, 299, 299, 3) dtype=float32>,)
(<tf.Tensor 'Sub/y:0' shape=() dtype=float32>,)
(<tf.Tensor 'Sub:0' shape=(1, 299, 299, 3) dtype=float32>,)
(<tf.Tensor 'Mul/y:0' shape=() dtype=float32>,)
(<tf.Tensor 'Mul:0' shape=(1, 299, 299, 3) dtype=float32>,)
(<tf.Tensor 'conv/conv2d_params:0' shape=(3, 3, 3, 32) dtype=float32>,)
(<tf.Tensor 'conv/Conv2D:0' shape=(1, 149, 149, 32) dtype=float32>,)
(<tf.Tensor 'conv/batchnorm/beta:0' shape=(32,) dtype=float32>,)
(<tf.Tensor 'conv/batchnorm/gamma:0' shape=(32,) dtype=float32>,)
(<tf.Tensor 'conv/batchnorm/moving_mean:0' shape=(32,) dtype=float32>,)
(<tf.Tensor 'c

In [ ]:
mixed_9/conv:0